In [ ]:
import pathlib
import usability

## Welche Daten für welche Hypothese?

### Effizienz

1. Klicks
2. Tastatur
2. NASA

### Zufriedenheit

1. QUESI
2. Feedback

## Pfad zu den Daten

In [ ]:
directory = pathlib.Path("data", "collection")

## `Sample` Objekt

In [ ]:
sample = usability.Sample(directory)

## Prä-Fragebogen

In [ ]:
sample.pre

## NASA

In [ ]:
sample.nasa

### Beispiel: Nur Werte für Spreadshirt anzeigen

In [ ]:
sample.nasa[sample.nasa["website"] == "spreadshirt"]

## QUESI

In [ ]:
sample.quesi

## Feedback

In [ ]:
sample.feedback

## Tastatur

In [ ]:
sample.keyboard

## Maus

In [ ]:
sample.mouse